In [0]:
import pandas as pd

In [0]:
import xml.etree.ElementTree as et

In [0]:
import os

In [0]:
import re

In [0]:
import urllib.request

In [0]:
xtree = et.parse(urllib.request.urlopen("https://raw.githubusercontent.com/nicklogin/NLPTeam3000/master/development/SentiRuEval_rest_train.xml"))

In [0]:
!wget https://github.com/nicklogin/NLPTeam3000/archive/master.zip

--2019-12-22 22:06:45--  https://github.com/nicklogin/NLPTeam3000/archive/master.zip
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/nicklogin/NLPTeam3000/zip/master [following]
--2019-12-22 22:06:45--  https://codeload.github.com/nicklogin/NLPTeam3000/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip.1’

master.zip.1            [     <=>            ]  11.71M  3.00MB/s    in 3.9s    

2019-12-22 22:06:50 (3.00 MB/s) - ‘master.zip.1’ saved [12277133]



In [0]:
!unzip master.zip

Archive:  master.zip
e147713f32a3e8a2c083bac0d49f4ca4993b09e9
replace NLPTeam3000-master/.ipynb_checkpoints/convert to needed format-checkpoint.ipynb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NLPTeam3000-master/.ipynb_checkpoints/convert to needed format-checkpoint.ipynb  
replace NLPTeam3000-master/.ipynb_checkpoints/find keywords-checkpoint.ipynb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NLPTeam3000-master/.ipynb_checkpoints/find keywords-checkpoint.ipynb  
replace NLPTeam3000-master/12943.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NLPTeam3000-master/12943.tsv  
replace NLPTeam3000-master/12943.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NLPTeam3000-master/12943.xlsx  
replace NLPTeam3000-master/13823.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NLPTeam3000-master/13823.tsv  
replace NLPTeam3000-master/13823.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NLPTeam3000-master/13823.xlsx  
replace NLPTeam3000-

In [0]:
os.listdir('NLPTeam3000-master/разметка_финал')

['20086.tsv',
 '.gitignore',
 '5648.tsv',
 '33693.tsv',
 '12943.tsv',
 '28083.tsv',
 '33591.tsv',
 '35486.tsv',
 '13823.tsv',
 '32840.tsv',
 '32856.tsv']

In [0]:
os.listdir('NLPTeam3000-master/conllu_data')

['20086.tsv',
 '5648.tsv',
 '33693.tsv',
 '12943.tsv',
 '28083.tsv',
 '33591.tsv',
 '35486.tsv',
 '13823.tsv',
 '32840.tsv',
 '32856.tsv']

In [0]:
root = xtree.getroot()

In [0]:
data = []
columns = ['id', 'food', 'service', 'text']

In [0]:
for review in root:
    text_id = int(review.attrib['id'])
    
    scores = review.find('scores')
    
    food = int(scores.find('food').text)
    service = int(scores.find('service').text)
    
    text = review.find('text').text
    
    data.append({'id': text_id,
                'food': food,
                'service': service,
                'text': text})
    

df = pd.DataFrame(data, columns=columns)
df = df.set_index('id')

In [0]:
df.head()

,food,service,text
id,,,
17600,8,8,И пускай на меня не обижается наш прославленны...
23518,9,10,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,..."
27221,9,1,"Советую вам уволить Вашего метродотеля Елену, ..."
29097,8,9,отличный средне вековый интеръер. Приятное обс...
23065,10,8,Ужинали в ресторане Баден-Баден 6 марта . Импо...


In [0]:
texts = df['text']
y = df[['food', 'service']].applymap(lambda x: 1 if x > 5 else 0)

In [0]:
len(df)

19034

In [0]:
y.head()

,food,service
id,,
17600,1,1
23518,1,1
27221,1,0
29097,1,1
23065,1,1


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

Для начала обучим простой классифаер текстов:

In [0]:
import numpy as np

In [0]:
vectorizer = TfidfVectorizer()

texts_train, texts_test, y_train, y_test = train_test_split(texts, y, test_size=0.2, random_state=42)

X_train = vectorizer.fit_transform(texts_train)
X_test = vectorizer.transform(texts_test)

parameter_grid = {"penalty": ["l1", "l2", "elasticnet"],
                 "alpha": [10**i for i in range(-10, 1)]}

clf_food = GridSearchCV(SGDClassifier(random_state=42, n_jobs=2, loss="log"), parameter_grid, n_jobs=2)
clf_food.fit(X_train, y_train["food"])
clf_food.score(X_test, y_test["food"])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.8360914105594957

In [0]:
clf_service = GridSearchCV(SGDClassifier(random_state=42, n_jobs=2, loss="log"), parameter_grid, n_jobs=2)
clf_service.fit(X_train, y_train["service"])
clf_service.score(X_test, y_test["service"])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.860782768584187

Продумаем простую реализацию поиска тональных слов - для каждого слова смотрим контексты, включающие его длины 5 и если во всех контекстах вероятность положительного или отрицательного класса больше определённого значения, считаем это слово тональным

In [0]:
clf_food.predict_proba(vectorizer.transform(["Невкусная еда"]))

array([[0.95116738, 0.04883262]])

In [0]:
clf_food.predict_proba(vectorizer.transform(['Вкусная еда']))

array([[0.00569557, 0.99430443]])

In [0]:
clf_service.predict_proba(vectorizer.transform(['Отличное обслуживание']))

array([[0.01318158, 0.98681842]])

In [0]:
clf_service.predict_proba(vectorizer.transform(['Ужасное обслуживание']))

array([[0.9945265, 0.0054735]])

Загрузим наши размеченные тексты:

In [0]:
def get_prepared_data(conllu_data_folder, tonal_folder):
    texts = []
    food_pos = []
    food_neg = []
    service_pos = []
    service_neg = []
    
    for file in os.listdir(conllu_data_folder):
        if file.endswith('.tsv'):
            sent_id = 0

            sentences = []
            
            text = open(os.path.join(conllu_data_folder, file), 'r', encoding='utf-8').read()

            for line in text.splitlines()[1:]:
                if '# sent_id' in line:
                    sent_id = re.search("sent_id = ([0-9]+)", line).group(1)
                    sentences.append([])
                try:
                    token_id, token, lemma, pos, *other = line.split('\t')
                except:
                    continue
                if token:
                    sentences[-1].append((sent_id, int(token_id), token))
            
            markup = open(os.path.join(tonal_folder, file[:-4]+".tsv"), 'r', encoding='utf-8').read()
            
            food_neg_ids = set()
            food_pos_ids = set()
            service_neg_ids = set()
            service_pos_ids = set()
            
            for line in markup.splitlines():
                line = line.strip()
                if line:
                    sent_id, token_ids, aspect, mark = line.split('\t')
                    token_ids = [int(i) for i in token_ids.split(',')]
                    token_ids = [i for i in range(token_ids[0], token_ids[-1])]
                    if aspect == 'Food' and mark == '1':
                        for token_id in token_ids:
                            food_pos_ids.add((sent_id, token_id))
                    elif aspect == 'Food' and mark == '0':
                        for token_id in token_ids:
                            food_neg_ids.add((sent_id, token_id))
                    elif aspect == 'Service' and mark == '1':
                        for token_id in token_ids:
                            service_pos_ids.add((sent_id, token_id))
                    elif aspect == 'Service' and mark == '0':
                        for token_id in token_ids:
                            service_neg_ids.add((sent_id, token_id))
                    
            for sent in sentences:
                for sent_id, token_id, token in sent:
                    for lst, hashmap in zip([food_neg, food_pos, service_neg, service_pos],
                        [food_neg_ids, food_pos_ids, service_neg_ids, service_pos_ids]):
                        if (sent_id, token_id) in hashmap:
                            lst.append(1)
                        else:
                            lst.append(0)

            for sent_id, sent in enumerate(sentences):
              for token_id, token in enumerate(sent):
                sentences[sent_id][token_id] = token[2]
            texts.append(sentences)
            
    return texts, food_pos, food_neg, service_pos, service_neg

In [0]:
data, food_pos, food_neg, service_pos, service_neg  = get_prepared_data(conllu_data_folder="NLPTeam3000-master/conllu_data",
                                                                       tonal_folder="NLPTeam3000-master/разметка_финал")

In [0]:
sum(food_pos)

9

In [0]:
def predict_text_proba(texts, clf, vectorizer):
    return clf.predict_proba(vectorizer.transform(texts))

In [0]:
avg = lambda x: sum(x)/len(x)

def extract_tonal_words(texts, proba_predictor, threshold=0.7, seq_len=3, criterion="min"):
    is_neg = []
    is_pos = []
    for text in data:
        for sent in text:
            for ind, word in enumerate(sent):
                seqs = [sent[i:i+seq_len] for i in range(ind-seq_len-1,ind)]
                seqs = [" ".join(seq) for seq in seqs]
                probas = proba_predictor(seqs)
                neg_probas = [i[0] for i in probas]
                pos_probas = [i[1] for i in probas]
                
                if criterion == "max":
                    neg_proba, pos_proba = max(neg_probas), max(pos_probas)
                elif criterion == 'min':
                    neg_proba, pos_proba = min(neg_probas), min(pos_probas)
                elif criterion == 'avg':
                    neg_proba, pos_proba = avg(neg_probas), avg(pos_probas)
                else:
                    raise Exception
                
                if neg_proba > threshold:
                    is_neg.append(1)
                else:
                    is_neg.append(0)
                
                if pos_proba > threshold:
                    is_pos.append(1)
                else:
                    is_pos.append(0)
            
    return is_pos, is_neg

Подберём наилучшие праметры для предложенного нами алгоритма (порог, длину последовательности, критерий):

In [0]:
food_predictor = lambda x: predict_text_proba(x, clf=clf_food, vectorizer=vectorizer)

In [0]:
food_pos_pred, food_neg_pred = extract_tonal_words(data, food_predictor, threshold=0.99)

In [0]:
from sklearn.metrics import f1_score, accuracy_score

In [0]:
f1_score(food_pos, food_pos_pred)

0.0

In [0]:
f1_score(food_neg, food_neg_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

In [0]:
from sklearn.model_selection import ParameterGrid

In [0]:
param_grid = {'threshold': [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.98, 0.99, 0.995],
              'seq_len': [i for i in range(3, 11)],
              'criterion': ['max', 'min', 'avg']}
param_grid = ParameterGrid(param_grid)

best_f1 = 0

history = []

best_params = None

for param_dict in param_grid:
  food_pos_pred, food_neg_pred = extract_tonal_words(data, food_predictor, **param_dict)
  f1_pos = f1_score(food_pos, food_pos_pred)
  f1_neg = f1_score(food_neg, food_neg_pred)
  f1 = (f1_pos + f1_neg)/2
  history.append((f1_pos, f1_neg, param_dict))
  if f1 > best_f1:
    best_f1 = f1
    best_params = param_dict

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
best_f1

0.125

In [0]:
best_params

{'criterion': 'min', 'seq_len': 9, 'threshold': 0.75}

In [0]:
food_pos_pred, food_neg_pred = extract_tonal_words(data, food_predictor, **best_params)

In [0]:
f1_score(food_pos, food_pos_pred)

0.02777777777777778

In [0]:
f1_score(food_neg, food_neg_pred)

0.2222222222222222

In [0]:
history

[(0.015088013411567479,
  0.004878048780487805,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.7}),
 (0.016981132075471694,
  0.006688963210702341,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.75}),
 (0.01756311745334797,
  0.00881057268722467,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.8}),
 (0.018205461638491547,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.85}),
 (0.016556291390728478,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.9}),
 (0.026595744680851064,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.95}),
 (0.019230769230769232,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.98}),
 (0.027586206896551724,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.99}),
 (0.037735849056603765,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.995}),
 (0.014754098360655735,
  0.005154639175257732,
  {'criterion': 'max', 'seq_len': 4, 'threshold': 0.7}),
 (0.01646843549862763,
  0.006968641114982579,


In [0]:
service_predictor = lambda x: predict_text_proba(x, clf=clf_service, vectorizer=vectorizer)

In [0]:
param_grid = {'threshold': [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.98, 0.99, 0.995],
              'seq_len': [i for i in range(3, 11)],
              'criterion': ['max', 'min', 'avg']}
param_grid = ParameterGrid(param_grid)

best_f1 = 0

history_service = []

best_params = None

for param_dict in param_grid:
  service_pos_pred, service_neg_pred = extract_tonal_words(data, service_predictor, **param_dict)
  f1_pos = f1_score(service_pos, service_pos_pred)
  f1_neg = f1_score(service_neg, service_neg_pred)
  history_service.append((f1_pos, f1_neg, param_dict))
  f1 = (f1_neg + f1_pos)/2
  if f1 > best_f1:
    best_f1 = f1
    best_params = param_dict

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
best_f1, best_params

(0.09090909090909091, {'criterion': 'min', 'seq_len': 6, 'threshold': 0.98})

In [0]:
history_service

[(0.011955593509820667,
  0.021834061135371178,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.7}),
 (0.013245033112582783,
  0.027472527472527472,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.75}),
 (0.015401540154015401,
  0.03663003663003663,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.8}),
 (0.015325670498084292,
  0.043859649122807015,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.85}),
 (0.015552099533437011,
  0.05319148936170212,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.9}),
 (0.026178010471204192,
  0.07766990291262134,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.95}),
 (0.034632034632034625,
  0.09523809523809523,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.98}),
 (0.027777777777777776,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.99}),
 (0.02531645569620253,
  0.0,
  {'criterion': 'max', 'seq_len': 3, 'threshold': 0.995}),
 (0.011824324324324325,
  0.022172949002217297,
  {'criterion': 'max', 'seq_len': 4, 't